In [1]:
import os

from google.colab import drive
drive.mount('/content/drive/')

os.chdir('/content/drive/MyDrive/Times-Series-Library/')
os.getcwd()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


'/content/drive/MyDrive/Times-Series-Library'

In [ ]:
!pip install -r requirements.txt

In [2]:
trials = os.listdir('./results')
print(*trials, sep='\n')

model_ID1_run_000@iter0
model_new_model_run_new_run_1@iter0
model_ID1_run_003@iter0


In [3]:
import numpy as np
import pandas as pd
import plotly.express as px

In [23]:
save_dir='results/' + trials[2] + '/test'
result_df = pd.read_csv(f'{save_dir}/result_df.csv', index_col=0)
result_df.index = pd.to_datetime(result_df.index)


In [24]:
def plot(df, title=None, save=False):
  fig = px.line(df, template='plotly_white')
  fig.update_yaxes(title_text='')
  fig.update_xaxes(title_text='')
  fig.update_layout(hovermode='x unified',
                    title_text=title,title_x=.5,
                    legend=dict(orientation="h", title_text=''))
  if save:
    fig.write_image(f"{save_dir}/{title}.png")
  return fig


In [26]:
plot(result_df, title='result')

Output hidden; open in https://colab.research.google.com to view.

In [33]:
clipped_df = result_df.copy(deep=True)


after = clipped_df.loc['2020-03-01':, :]
clipped_df.loc[:, 'prediction'] = np.clip(clipped_df.prediction.values, after.target.min(), after.target.max())


plot(clipped_df, title='clipped result')


Output hidden; open in https://colab.research.google.com to view.

In [28]:
error_df = pd.concat([result_df.iloc[:, 1] - result_df.iloc[:, 0],
                      clipped_df.iloc[:, 1] - clipped_df.iloc[:, 0]],
                     axis='columns').dropna()
error_df.columns = ['min-level output', '30 mins averange output']
plot(error_df, title='error')

Output hidden; open in https://colab.research.google.com to view.

In [29]:
error_pct_df = error_df.copy(deep=True)
error_pct_df.iloc[:,0]=error_pct_df.iloc[:,0].values / result_df.dropna().target.values
error_pct_df.iloc[:,1]=error_pct_df.iloc[:,0].values / result_df.dropna().target.values
fig = plot(error_pct_df, title='error percentage')
fig.update_yaxes(tickformat = '.0%')

Output hidden; open in https://colab.research.google.com to view.

In [30]:
plot(result_df.resample('30min').mean(), title='30 mins averange result')

Output hidden; open in https://colab.research.google.com to view.

In [34]:
plot(clipped_df.resample('30min').mean(), title='clipped 30 mins averange result')

Output hidden; open in https://colab.research.google.com to view.